# Tutorial 1: calibration

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
url = 'https://raw.githubusercontent.com/oguerrer/ppi_app/main/code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

url = 'https://raw.githubusercontent.com/oguerrer/ppi_app/main/code/calibrator.py'
r = requests.get(url)
with open('calibrator.py', 'w') as f:
    f.write(r.text)
import calibrator

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tequilamambo/Documents/GitHub/ppi_app/code/tutor/code/'